In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
df = pd.read_excel('./owid-covid-data.xlsx')

In [9]:
korea = df[df['iso_code']=='KOR']
korea = korea.fillna(0)

date = pd.DataFrame({'일자' : korea['date']})
location = pd.DataFrame({'location' : korea['location']})

korea.drop(['iso_code','continent','location','date'], axis=1,inplace=True)

In [10]:
X = korea[['total_cases',
 'excess_mortality_cumulative_absolute',
 'icu_patients',
 'total_vaccinations',
 'stringency_index',
 'new_cases',
 'weekly_hosp_admissions',
 'weekly_icu_admissions',
 'positive_rate',
 'reproduction_rate',
'new_tests',
'total_tests']]
y = korea['new_deaths']

In [11]:
lm = sm.OLS(y, X).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             new_deaths   R-squared (uncentered):                   0.932
Model:                            OLS   Adj. R-squared (uncentered):              0.931
Method:                 Least Squares   F-statistic:                              942.9
Date:                Wed, 11 May 2022   Prob (F-statistic):                        0.00
Time:                        19:38:48   Log-Likelihood:                         -3669.4
No. Observations:                 838   AIC:                                      7363.
Df Residuals:                     826   BIC:                                      7420.
Df Model:                          12                                                  
Covariance Type:            nonrobust                                                  
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
total_cases                           4.344e-06   6.03e-07      7.209      0.000    3.16e-06    5.53e-06
excess_mortality_cumulative_absolute    -0.0016      0.001     -2.881      0.004      -0.003      -0.001
icu_patients                             0.0656      0.005     12.989      0.000       0.056       0.076
total_vaccinations                   -2.895e-07      4e-08     -7.241      0.000   -3.68e-07   -2.11e-07
stringency_index                         0.0096      0.035      0.279      0.780      -0.058       0.077
new_cases                                0.0006   2.52e-05     23.934      0.000       0.001       0.001
weekly_hosp_admissions                   0.0058      0.001      9.009      0.000       0.005       0.007
weekly_icu_admissions                   -0.0090      0.007     -1.359      0.175      -0.022       0.004
positive_rate                           -3.7490      2.388     -1.570      0.117      -8.436       0.938
reproduction_rate                       -2.1515      1.422     -1.513      0.131      -4.942       0.639
new_tests                               -0.0001      1e-05    -10.603      0.000      -0.000   -8.65e-05
total_tests                           3.763e-07   2.37e-07      1.588      0.113   -8.87e-08    8.41e-07
==============================================================================
Omnibus:                      264.811   Durbin-Watson:                   1.751
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            15116.021
Skew:                          -0.566   Prob(JB):                         0.00
Kurtosis:                      23.776   Cond. No.                     1.98e+08
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.98e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [12]:
p = lm.predict(X).round(0)
p_dead = pd.DataFrame({'추정사망(명)':p})
p_dead[p_dead['추정사망(명)']<=0] = 0
dead = pd.DataFrame({'사망(명)':y})

ko_COVID = pd.concat([date,location,dead,p_dead], axis=1)
ko_COVID.reset_index(inplace=True)
ko_COVID.drop(['index'],axis=1,inplace=True)

ko_COVID.to_excel('./한국.xlsx')